# Boost Switching regulator LM3478
- Supply: 20 V
- Output voltage: 40 V (theoretically up to 45 V)
	- Make the output voltage variable through some variable resistor (Manually / voltage controlled)

In [ ]:

import math
#############! DESIGN PARAMETERS ##################
### Take from LTSpice simulation + add on top
I_peak_A = 2.5 # Amps, for each bridge spike of 1 amp 180 degrees shifted
I_rms_A = math.sqrt(I_peak_A) # Amps (slightly overdimensioned already)
print(f"I_rms_A: {I_rms_A}")

### DIODE Choice:
# 40 V max
# RMS voltage max: 28 V
# 5 A RMS current max
V_diode = 0.85 # SS210 (Max current: 2 Amps, Max voltage: 100 V)
V_max_diode = 100 # 

### MOSFET Choice: AO3400A-mosfet
# AO3400A-mosfet -> To low Vds
# NCE6005AS-mosfet -> usable in full bridge as well as sepic + MUCH cheaper
Vfet_gsth = 1.6 # Volt
Vfet_diode = 1.2 # Volt
Rfet_dsn = 32e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_rms_A
Vds_max = 60 # V
Vgs_max = 20 # V
Id_max = 5 # A
Id_peak = 24 # A (pulsed drain current)
print(f"Vfet_dsn {Vfet_dsn}")

# Required Output Voltage
Vout = 10

I_rms_A: 1.5811388300841898
Vfet_dsn 0.05059644256269407
